<a href="https://colab.research.google.com/github/sayakgis/Air_Quality/blob/master/Crack_train_on_cowi_data_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training ResNET34 on COWI DDG images


In [0]:
#GPU availability
!nvidia-smi

In [0]:
# Import basic libraries
import matplotlib.pyplot as plt
import numpy as np
import os,shutil

# Fastai reloads
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#Impost fastai
from fastai.vision import *
from fastai.metrics import error_rate,accuracy
from fastai.callbacks import *
from fastai.imports import *

from pathlib import Path

In [0]:
np.random.seed(42)

In [0]:
#download the data
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-datasets/180565/406017/data.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1557028882&Signature=ZQbUSiI7G9LK%2BKWd5%2BmoRJHiemRapKk%2F12pVxCI4M%2FV7ngTarjtlbHds5j64YvBmrmXwKznyIym7mzMcFMYX0LzK9VEqesP%2FmNC2vja5oFhHzc5KRLRzzunCEJFZZ9EEtboEB9VtP6tN1VSMc7ZldGrfd4iL0u6SlttLo4n0yn9Pcfsb2htPK%2B0wV%2By8MvZ3T3RSEATkBzoEAbGa6qcq8r8opWtae08ePwlnUwViGixu92pnRuTnPd5M4jAczpE4utI%2BVWSszqlrZ6KseVXAp9l1Avo3%2BYXr8ajbrte2kgZVMW9y1SBgAWmaa9Vh8MYAQr1%2FWKizi%2Ft3gpm3YbvOHQ%3D%3D" -O "data.zip" -c

In [0]:
!zipinfo -1 data.zip

In [0]:
#unzip the folder
!unzip data.zip

In [0]:
!ls data/Crack_Detection

In [0]:
#set path and size
PATH="data/Crack_Detection/"
os.listdir(PATH)

In [0]:
# Create validation dataset
#np.random.seed(12)
pos=os.listdir(f'{PATH}Positive')
neg=os.listdir(f'{PATH}Negative')

#take 20%: 4000*2 in validation dataset
val_pos=list(np.random.choice(pos,int(len(pos)/5),replace=False))
val_neg=list(np.random.choice(neg,int(len(neg)/5),replace=False))

#Create test dataset, take 3000*2 examples from validation datatset
test_pos=list(np.random.choice(val_pos,int(len(val_pos)/2),replace=False))
test_neg=list(np.random.choice(val_neg,int(len(val_neg)/2),replace=False))

print('First 5 files of positive examples:',pos[:5])
print('First 5 examples of validation set of positive examples:',val_pos[:5])
#val_neg[:5]
print('Total number of images in positive example:',len(pos))
print('Total number of images in negative example:',len(neg))

print('Total number of images in TEST negative example:',len(test_neg))
print('Total number of images in TEST positive example:',len(test_pos))

In [0]:
# remove
for i in val_pos:
  pos.remove(i)
  
for i in val_neg:
  neg.remove(i)
  
for i in test_pos:
  val_pos.remove(i)

for i in test_neg:
  val_neg.remove(i)

In [0]:
print('Total number of independent training examples:',len(pos)+len(neg))
print('Number of validation examples:',len(val_pos)+len(val_neg))
print('Number of test examples:',len(test_pos)+len(test_neg))

In [0]:
#create folder structute for fastai
os.makedirs(PATH+'train',exist_ok=True)
os.makedirs(PATH+'valid',exist_ok=True)
os.makedirs(PATH+'test',exist_ok=True)
os.makedirs(PATH+'train/Positive',exist_ok=True)
os.makedirs(PATH+'train/Negative',exist_ok=True)
os.makedirs(PATH+'valid/Positive',exist_ok=True)
os.makedirs(PATH+'valid/Negative',exist_ok=True)
#os.makedirs(PATH+'test/Positive',exist_ok=True)
#os.makedirs(PATH+'test/Negative',exist_ok=True)

In [0]:
# Move the positive examples to test folder
path = f'{PATH}Positive/'
moveto = f'{PATH}test/'
files = test_pos
#files.sort()
for f in files:
    src = path+f
    dst = moveto+'p'+f
   # print(f)
    shutil.move(src,dst)
    
# Move the negative examples to test folder
path = f'{PATH}Negative/'
moveto = f'{PATH}test/'
files = test_neg
#files.sort()
for f in files:
    src = path+f
    dst = moveto+'n'+f   #added n just to keep all files in same folder as the filename of rpostive and egatives are same
    #print(f)
    shutil.move(src,dst)

In [0]:
# Move the positive examples to validation folder
path = f'{PATH}Positive/'
moveto = f'{PATH}valid/Positive/'
files = val_pos
#files.sort()
for f in files:
    src = path+f
    dst = moveto+f
   # print(f)
    shutil.move(src,dst)
    
# Move the negative examples to validation folder
path = f'{PATH}Negative/'
moveto = f'{PATH}valid/Negative/'
files = val_neg
#files.sort()
for f in files:
    src = path+f
    dst = moveto+f
    #print(f)
    shutil.move(src,dst)

In [0]:
#move remaining positive files to train folder
path = f'{PATH}Positive/'
moveto = f'{PATH}train/Positive/'
files = os.listdir(path)
#files.sort()
for f in files:
    src = path+f
    dst = moveto+f
   # print(f)
    shutil.move(src,dst)
    
#move remaining negative files to train folder
path = f'{PATH}Negative/'
moveto = f'{PATH}train/Negative/'
files = os.listdir(path)
#files.sort()
for f in files:
    src = path+f
    dst = moveto+f
   # print(f)
    shutil.move(src,dst)

In [0]:
!ls data/Crack_Detection/Negative/

In [0]:
#Resize train
path =Path('data/Crack_Detection/')
path_hr = path/'train'
path_lr = path/'small-227'
path_mr = path/'small-299'
path_mhr=path/'small-350'

il = ImageList.from_folder(path_hr)

def resize_one(fn, i, path, size):
    dest = path/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, size, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
    img.save(dest, quality=75)

# create smaller image sets the first time this nb is run
sets = [(path_lr, 227), (path_mr, 299), (path_mhr, 350)]
for p,size in sets:
    if not p.exists(): 
        print(f"resizing to {size} into {p}")
        parallel(partial(resize_one, path=p, size=size), il.items)

In [0]:
#Resize test
path =Path('data/Crack_Detection/')
path_hr = path/'test'
path_lr = path/'test-small-227'
path_mr = path/'test-small-299'
path_mhr=path/'test-small-350'

il = ImageList.from_folder(path_hr)

def resize_one(fn, i, path, size):
    dest = path/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, size, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
    img.save(dest, quality=75)

# create smaller image sets the first time this nb is run
sets = [(path_lr, 227), (path_mr, 299), (path_mhr, 350)]
for p,size in sets:
    if not p.exists(): 
        print(f"resizing to {size} into {p}")
        parallel(partial(resize_one, path=p, size=size), il.items)

In [0]:
#Resize valid
path =Path('data/Crack_Detection/')
path_hr = path/'valid'
path_lr = path/'valid-small-227'
path_mr = path/'valid-small-299'
path_mhr=path/'valid-small-350'

il = ImageList.from_folder(path_hr)

def resize_one(fn, i, path, size):
    dest = path/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, size, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
    img.save(dest, quality=75)

# create smaller image sets the first time this nb is run
sets = [(path_lr, 227), (path_mr, 299), (path_mhr, 350)]
for p,size in sets:
    if not p.exists(): 
        print(f"resizing to {size} into {p}")
        parallel(partial(resize_one, path=p, size=size), il.items)

In [0]:
len(os.listdir('data/Crack_Detection/test-small-227'))

In [0]:
os.listdir(PATH)

**Create data loader with 227 size image**

In [0]:
data = ImageDataBunch.from_folder(PATH,train='small-227',valid='valid-small-227',test='test-small-227',ds_tfms=get_transforms(), 
                                  size=227, bs=256).normalize(imagenet_stats)

In [0]:
data.show_batch(3,figsize=(15,12))

**Load the pretrained model of public dataset**

In [0]:
learn=load_learner(PATH)

**Check pretrained weights**

In [0]:
learn.model.state_dict()

**Check architechture of the model**

In [0]:
learn.model

In [0]:
learn.model_dir

**Attach data loaded of DDG images at 227 size to the model**

In [0]:
learn.data=data

**Check classes of the data**

In [0]:
learn.data.classes

**Finding the optimal learning rate**

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

Let us set **Learning rate to 1e-1**

**Run 5 epochos and see **

In [0]:
lr=1e-1
learn.fit_one_cycle(5,slice(lr))

In [0]:
learn.recorder.plot_losses()
plt.title('Plot of Loss function')

Seeing the loss in validation set compared to training set it looks that the model is underfitting

**Plot accuracy metric and error with learning rate**

In [0]:
learn.recorder.plot_metrics()

It is seen that the model is improving even at epoch 5 so let us train for 2 more epochs and see

In [0]:
learn.fit_one_cycle(2,slice(lr))

Now the validation loss and training loss is very close

In [0]:
learn.recorder.plot_lr()

**Let us see the results on validation set**

In [0]:
#@title
learn.show_results(3)

**See top most misclassifications**

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
#@title
interp.plot_top_losses(25, figsize=(25,18),heatmap=False)

In [0]:
interp.plot_top_losses(25, figsize=(25,18),heatmap=True)

In [0]:
interp.plot_confusion_matrix(title='Confusion matrix on Validation dataset-227 size')

In [0]:
interp.most_confused()

In [0]:
#save the model
learn.save('stage-1-model-ddgimg')

In [0]:
#export the model for inference
learn.export('export-ddg.pkl')

In [0]:
#save the weights and bias
torch.save(learn.model.state_dict(),'data/Crack_Detection/models/stage-1-wts-resnet34-ddgimg.pth')

In [0]:
!ls data/Crack_Detection/models/

**Check prediction on Test dataset**

In [0]:
len(learn.data.test_ds)

In [0]:
#turn the model in evaluation mode
learn.model.eval()

In [0]:
#take random images from test dataset

plt.figure(figsize=(18,12))
plt.subplots_adjust(hspace=0.3,wspace=0.2)

n=np.random.randint(0,len(learn.data.test_ds),9)

for i,j in enumerate(n):
  img=data.test_ds[j][0]
  plt.subplot(3,3,i+1)
  plt.imshow(np.rollaxis(img.data.numpy(),0,3))
  pred,_,y=learn.predict(img)
  if ((Path(learn.data.test_ds.x.items[j]).name[0])=='n'):
         actual='Actual: Negative'
  else:
         actual='Actual: Positive'
  plt.title(actual
      +'\nPrediction: '+str(pred)+'-'+str(np.round(y.numpy().max()*100,2))+'%')
  plt.axis('Off')

In [0]:
test_preds,y,test_loss=learn.get_preds(ds_type=DatasetType.Test,with_loss=True)

In [0]:
test_img_names = [Path(i).name for i in learn.data.test_ds.x.items]

In [0]:
len(test_img_names)

In [0]:
test_probs_df = pd.DataFrame(test_preds.numpy(), columns=learn.data.train_ds.classes)
test_probs_df['image_name'] = test_img_names
test_probs_df.head()

In [0]:
test_probs_df['pred_label']= test_preds.numpy().argmax(1)
test_probs_df.head()

In [0]:
test_probs_df['true_label']=test_probs_df.image_name.apply(lambda x: 1 if x[0]=='p' else 0)
test_probs_df.head()

In [0]:
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report
import seaborn as sns

conf=confusion_matrix(y_pred=test_probs_df.pred_label,y_true=test_probs_df.true_label)

sns.heatmap(conf,fmt='d',annot=True,cmap='gray_r',xticklabels=['Negative','Positive'],yticklabels=['Negative','Positive'],square=True,linecolor='black',linewidths=1)
plt.title('Confusion Matrix on Test dataset of 227 images size')
plt.xlabel('Predicted')
plt.ylabel('Actual')

print('Accuracy of prediction on test images:',np.round(100*accuracy_score(y_pred=test_probs_df.pred_label,y_true=test_probs_df.true_label),2),'%')
print('\nClassification report:\n',classification_report(y_pred=test_probs_df.pred_label,y_true=test_probs_df.true_label))

**See the false cases**

In [0]:
false_case=test_probs_df[test_probs_df.pred_label!=test_probs_df.true_label]
false_case.head()

**Visualise the wrong classifications**

In [0]:
plt.figure(figsize=(16,35))
plt.subplots_adjust(hspace=0.2,wspace=0.1)

for i in range(false_case.shape[0]):
  plt.subplot(9,5,i+1)
  #print(false_case.iloc[i][0])
  plt.imshow(plt.imread(PATH+'test-small-227/'+false_case.iloc[i][2]))
  plt.title('True label: '+str(false_case.iloc[i][4])+' \n Predicted: '+str(false_case.iloc[i][3])+', Prob:'+str(np.round(np.max(false_case.iloc[i][:2])*100,2))+'%')
  plt.axis('Off')

## GradCAM implimentation

In [0]:
from fastai.callbacks.hooks import *
import cv2
import matplotlib.pyplot as plt
import random

# hook into forward pass
def hooked_backward(m, oneBatch, cat):
    # we hook into the convolutional part = m[0] of the model
    with hook_output(m[0]) as hook_a: 
        with hook_output(m[0], grad=True) as hook_g:
            preds = m(oneBatch)
            preds[0,int(cat)].backward()
    return hook_a,hook_g

# We can create a utility function for getting a validation image with an activation map
def getHeatmap(val_index, learner, imgDataBunch):
    """Returns the validation set image and the activation map"""
    # this gets the model
    m = learner.model.eval()
    tensorImg,cl = imgDataBunch.valid_ds[val_index]
    # create a batch from the one image
    oneBatch,_ = imgDataBunch.one_item(tensorImg)
    oneBatch_im = vision.Image(imgDataBunch.denorm(oneBatch)[0])
    # convert batch tensor image to grayscale image with opencv
    cvIm = cv2.cvtColor(image2np(oneBatch_im.data), cv2.COLOR_RGB2GRAY)
    # attach hooks
    hook_a,hook_g = hooked_backward(m, oneBatch, cl)
    # get convolutional activations and average from channels
    acts = hook_a.stored[0].cpu()
    #avg_acts = acts.mean(0)

    # Grad-CAM
    grad = hook_g.stored[0][0].cpu()
    grad_chan = grad.mean(1).mean(1)
    grad.shape,grad_chan.shape
    mult = (acts*grad_chan[...,None,None]).mean(0)
    return mult, cvIm

# Then, modify our plotting func a bit
def plot_heatmap_overview(interp:ClassificationInterpretation, learner, imgDataBunch, classes=data.classes):
    # top losses will return all validation losses and indexes sorted by the largest first
    tl_val,tl_idx = interp.top_losses()
    #classes = interp.data.classes
    fig, ax = plt.subplots(3,4, figsize=(16,12))
    fig.suptitle('Grad-CAM\nPredicted / Actual / Loss / Probability',fontsize=20)
    # Random
    for i in range(4):
        random_index = random.randint(0,len(tl_idx))
        idx = tl_idx[random_index]
        act, im = getHeatmap(idx, learner, imgDataBunch)
        H,W = im.shape
        _,cl = interp.data.dl(DatasetType.Valid).dataset[idx]
        cl = int(cl)
        ax[0,i].imshow(im)
        ax[0,i].imshow(im, cmap=plt.cm.gray)
        ax[0,i].imshow(act, alpha=0.5, extent=(0,H,W,0),
              interpolation='bilinear', cmap='inferno')
        ax[0,i].set_xticks([])
        ax[0,i].set_yticks([])
        ax[0,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[0,0].set_ylabel('Random samples', fontsize=16, rotation=0, labelpad=80)
    # Most incorrect or top losses
    for i in range(4):
        idx = tl_idx[i]
        act, im = getHeatmap(idx, learner, imgDataBunch)
        H,W = im.shape
        _,cl = interp.data.dl(DatasetType.Valid).dataset[idx]
        cl = int(cl)
        ax[1,i].imshow(im)
        ax[1,i].imshow(im, cmap=plt.cm.gray)
        ax[1,i].imshow(act, alpha=0.5, extent=(0,H,W,0),
              interpolation='bilinear', cmap='inferno')
        ax[1,i].set_xticks([])
        ax[1,i].set_yticks([])
        ax[1,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[1,0].set_ylabel('Most incorrect\nsamples', fontsize=16, rotation=0, labelpad=80)
    # Most correct or least losses
    for i in range(4):
        idx = tl_idx[len(tl_idx) - i - 1]
        act, im = getHeatmap(idx, learner, imgDataBunch)
        H,W = im.shape
        _,cl = interp.data.dl(DatasetType.Valid).dataset[idx]
        cl = int(cl)
        ax[2,i].imshow(im)
        ax[2,i].imshow(im, cmap=plt.cm.gray)
        ax[2,i].imshow(act, alpha=0.5, extent=(0,H,W,0),
              interpolation='bilinear', cmap='inferno')
        ax[2,i].set_xticks([])
        ax[2,i].set_yticks([])
        ax[2,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[2,0].set_ylabel('Most correct\nsamples', fontsize=16, rotation=0, labelpad=80)

In [0]:
plot_heatmap_overview(interp,learn,data)